In [10]:
import requests
import pandas as pd

In [11]:
api_key = 'af196f0478mshd90862dc657e127p18d90bjsn598fe178eab7'
year1 = 2012
year2 = 2014
genre = 'Drama'

In [12]:
class RecolectorPeliculas:
    def __init__(self, api_key):
        self.base_url = "https://moviesdatabase.p.rapidapi.com/titles"
        self.headers = {
            "X-RapidAPI-Key": api_key,
            "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
        }

    def llamar_pelis_api(self, year1, year2, genre):
        datos_paginas = []
        querystring = {"genre": genre, "startYear": year1, "titleType": "movie", "endYear": year2}
        page = 1

        while True:
            response = requests.get(self.base_url, headers=self.headers, params=querystring)
            if response.status_code == 200:
                json_pelis = response.json()
                datos_paginas.extend(json_pelis['results'])
                if 'next' in json_pelis and json_pelis['next']:
                    querystring['page'] = page + 1
                else:
                    break
            else:
                print("Error en la solicitud:", response.status_code, response.reason)
                return None

        return {"results": datos_paginas}

    def limpiar_json(self, datos_json):
        pelis = []
        for movie in datos_json['results']:
            type = movie.get('titleType', {}).get('text', 'No disponible')
            name = movie.get('titleText', {}).get('text', 'No disponible')
            year = movie.get('releaseYear', {}).get('year', 'No disponible')
            month = movie.get('releaseDate', {}).get('month', 'No consta') if movie.get('releaseDate') else 'No consta'
            id_movie = movie.get('id', 'No disponible')
            
            pelis.append({
                'Tipo': type,
                'Nombre': name,
                'Año de Estreno': year,
                'Mes de Estreno': month,
                'ID': id_movie
            })
        return pd.DataFrame(pelis)

    def conseguir_datos_pelis(self, year1, year2, genre):
        datos = self.llamar_pelis_api(year1, year2, genre)
        if datos:
            return self.limpiar_json(datos)
        else:
            return None

def convertir_csv(df, nombrecsv):
    df.to_csv(nombrecsv, index=False)

In [13]:
recolector = RecolectorPeliculas(api_key)
    
df_pelis = recolector.conseguir_datos_pelis(year1, year2, genre)

Error en la solicitud: 429 Too Many Requests


In [ ]:
   
if df_pelis is not None:
    nombre_csv = "Películas_Drama_2011-2012.csv"  
    convertir_csv(df_pelis, nombre_csv)
    print(f"Datos guardados exitosamente en {nombre_csv}")
else:
    print("No ha podido obtener datos de la API.")